In [ ]:
import os, sys 
#to be able to interact with Google Drive's operating system
from google.colab import drive 
#drive is a module that allows us use Python to interact with google drive
drive.mount('/content/gdrive') 
#mounting google drive allows us to work with its contents
nb_path = '/content/notebooks'
os.symlink('/content/gdrive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)
#The last three lines are what changes the path of the file.c

Mounted at /content/gdrive


Install the required package

In [ ]:
pip install dcor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/jakobrunge/tigramite.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jakobrunge/tigramite.git to /tmp/pip-req-build-tbxd2ept
  Running command git clone -q https://github.com/jakobrunge/tigramite.git /tmp/pip-req-build-tbxd2ept
     |████████████████████████████████| 15.7 MB 30.2 MB/s 
ERROR: Could not find a version that satisfies the requirement scipy==1.8.0 (from tigramite) (from versions: 0.8.0, 0.9.0, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.16.0, 0.16.1, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.3.0rc1, 1.3.0rc2, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.4.0rc1, 1.4.0rc2, 1.4.0, 1.4.1, 1.5.0rc1, 1.5.0rc2, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 1.5.4, 1.6.0rc1, 1.6.0rc2, 1.6.0, 1.6.1, 1.6.2, 1.6.3, 1.7.0rc1, 1.7.0rc2, 1.7.0, 1.7.1, 1.7.2, 1.7.3)
ERROR: No matching distribution found for scipy==1.8.0


In [ ]:
!pip install tigramite==4.2.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 420 kB 29.6 MB/s 


Import requied libraries

In [ ]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
#import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

/content/notebooks/tigramite/independence_tests/gpdc.py:16: UserWarning: No module named 'dcor'
  warnings.warn(str(e))
/content/notebooks/tigramite/independence_tests/gpdc_torch.py:18: UserWarning: No module named 'dcor'
  warnings.warn(str(e))


Upload the dataset we are interested to find the lagged causal links

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving X4_4.csv to X4_4.csv


In [ ]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['X4_4.csv']))# Dataset is now stored in a Pandas Dataframe

In [ ]:
df=df.to_numpy()
print(df)

[[0.53553229 1.06300183 1.6551918  1.09968929]
 [0.34367353 0.74225673 1.02323071 1.26927655]
 [1.026257   0.79815846 1.99724851 1.46383566]
 ...
 [0.96865376 1.85661336 7.76185468 3.91299906]
 [1.43355836 2.1075005  7.00175109 3.81140771]
 [0.87596712 2.10074539 6.88335661 3.93023553]]


In [ ]:
# Initialize dataframe object, specify time axis and variable names
var_names = [r'$X^1$', r'$X^2$', r'$X^3$', r'$X^4$']
dataframe = pp.DataFrame(df, 
                         datatime = np.arange(len(df)), 
                         var_names=var_names)

Perform the test using the parameters: significance level of the CI tests, minimum and maximum period of lags. Record the lagged links for the further steps.

In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=0, tau_max=4, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 2 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.077
        ($X^3$  0): pval = 0.01200 | val =  0.010 | unoriented link

    Variable $X^2$ has 3 link(s):
        ($X^2$ -1): pval = 0.01400 | val =  0.024
        ($X^1$ -1): pval = 0.00000 | val =  0.023
        ($X^2$ -2): pval = 0.00600 | val =  0.019

    Variable $X^3$ has 2 link(s):
        ($X^2$ -2): pval = 0.00000 | val =  0.020
        ($X^1$  0): pval = 0.01200 | val =  0.010 | unoriented link

    Variable $X^4$ has 1 link(s):
        ($X^3$  0): pval = 0.00000 | val =  0.041

## Ambiguous triples:

    ($X^2$ -2) --> $X^3$ o-o $X^1$
    ($X^4$  0) o-o $X^3$ o-o $X^1$
    ($X^1$ -1) --> $X^1$ o-o $X^3$
    ($X^1$  0) o-o $X^3$ o-o $X^4$


In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=1, tau_max=4, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 1 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.089

    Variable $X^2$ has 3 link(s):
        ($X^2$ -1): pval = 0.01500 | val =  0.024
        ($X^1$ -1): pval = 0.00000 | val =  0.023
        ($X^2$ -2): pval = 0.00300 | val =  0.019

    Variable $X^3$ has 2 link(s):
        ($X^2$ -2): pval = 0.00000 | val =  0.037
        ($X^3$ -3): pval = 0.00000 | val =  0.023

    Variable $X^4$ has 1 link(s):
        ($X^2$ -2): pval = 0.00100 | val =  0.027
